# Optimizing Performance Using Numba & Cython

## Numba & Cython: What are they?
At a high level, Numba and Cython are both modules that make your Python code run faster. This means we can have the quick prototyping and iteration that Python is known for, while getting the speed we expect from programs written in C. This is great--we can have our cake and eat it too!

## Use Case: Cholesky Decomposition
Matrix computations are a standard benchmark for speed. In that vein, we'll be examining the execution time of various implementations of Cholesky Decomposition, a method of matrix decomposition that we used in a previous homework assignment:
* A pure Python implementation
* A Numba-fied implementation
* A Cython-ized implementation
* The provided SciPy implementation

We'll begin with a pure Python implementation and work our way towards the Numba-fied and Cython-ized versions. Then, we'll see how those fare against SciPy.

## Some mathematical formalism:
This isn't too important for implementing a way of calculating the Cholesky decomposition, but it might provide some intuition. Feel free to skip ahead to the next section if you don't want to deal with the linear algebra details.

Formally, the Cholesky decomposition (factorization) is the decomposition of a Hermitian, positive-definite matrix into the product of a lower triangular matrix and its conjugate transpose. There a lot of terms here that might be foreign if you've never taken a course in linear algebra before, so I'll try to break them down:

1. A square matrix $A$ is said to be Hermitian if for every entry $a_{i,j}$ in $A$, it's true that $a_{i,j} = \overline{a}_{j,i}$, where $\overline{z}$ denotes the complex conjugate of $z$.
    - The complex conjugate of a complex number $z = a + bi\,$ is defined to be $\overline{z} := a - bi$.
    - Symmetric, real, square matrices are Hermitian
2. A Hermitian matrix $A$ is said to be positive-definite if the scalar $\overline{z} A z$ is real and positive for all non-zero column vectors $z$ of complex numbers.
3. A lower triangle matrix is a matrix $L$ of the form $$\begin{bmatrix}\ell_{11} & 0 & \cdots & 0 \\ \ell_{21} & \ell_{22} & \cdots & 0 \\ \vdots & \vdots & \ddots & 0 \\ \ell_{n1} & \ell_{n2} & \cdots & \ell_{nn}\end{bmatrix}$$

So, given a symmetric, positive-definite matrix $A$, the Cholesky decomposition of $A$ gives us a lower triangular matrix $L$ such that 
$$
    A = \begin{bmatrix}
            a_{11} & a_{12} & \cdots & a_{1n} \\
            a_{21} & a_{22} & \cdots & a_{2n} \\
            \vdots & \vdots & \ddots & \vdots \\
            a_{n1} & a_{n2} & \cdots & a_{nn}
       \end{bmatrix} 
       =
       \begin{bmatrix}
           \ell_{11} & 0 \cdots & 0\\
           \ell_{21} & \ell_{22} \cdots & 0\\
           \vdots & \vdots & \ddots & 0\\
           \ell_{n1} & \ell_{n2} & \cdots & \ell_{nn}
       \end{bmatrix}
        \begin{bmatrix}
           \overline{\ell}_{11} & \overline{\ell}_{21} & \cdots & \overline{\ell}_{n1} \\
           0 & \overline{\ell}_{22} & \cdots & \overline{\ell}_{n2}\\
           \vdots & \vdots & \ddots & \vdots\\
           0 & 0 & \cdots & \overline{\ell}_{nn}
       \end{bmatrix}
        = L\overline{L}^T
$$

## The Cholesky-Banachiewicz  & Cholesky-Crout Algorithms
###### (Wikipedia)
Let $A$ be a real, symmetric, positive-definite matrix. Then, the Cholesky factor $L$ of $A$ is the lower triangular matrix such that $A = LL^T$ and
$$
    L_{ii} = \sqrt{A_{ii} - \sum_{k=1}^{i-1}L_{i,k}^2}
    \\
    L_{ij} = \frac{1}{L_{jj}}\left(A_{ij} - \sum_{k=1}^{j-1}L_{ik}L_{jk}\right) \;\;\; \text{for $i > j$}
$$
This means that we can compute $L_{ij}$ if we know the entries to the left and above. 
- The Cholesky-Banachiewicz algorithm starts from the upper left corner of the matrix $L$ and proceeds to calculate the matrix row by row.
- The Cholesky-Crout algorithm starts from the upper left corner of the matrix $L$ and proceeds to calculate the matrix column by column.

Now we can move on to implementation.

In [1]:
def cholesky_banachiewicz_pure(A):
    n = len(A)
    L = [[0.0] * n for _ in xrange(n)]
    
    for i in xrange(n):
        for j in xrange(i + 1): # Build row i
            temp = A[i][j] - sum(L[i][k] * L[j][k] for k in xrange(j))
            L[i][j] = temp**0.5 if i == j else temp / L[j][j]
            
    return L

def cholesky_crout_pure(A):
    n = len(A)
    L = [[0.0] * n for _ in xrange(n)]
    
    for j in xrange(n):
        for i in xrange(j, n): # Build column i
            temp = A[i][j] - sum(L[i][k] * L[j][k] for k in xrange(j))
            L[i][j] = temp**0.5 if i == j else temp / L[j][j]
    
    return L

Let's consider a small example just to verify that our implementation is working as intended.

In [2]:
import scipy
import scipy.linalg

A = [[6, 3, 4, 8], [3, 6, 5, 1], [4, 5, 10, 7], [8, 1, 7, 25]]
A_array = scipy.array(A)

L_banachiewicz = cholesky_banachiewicz_pure(A)
L_crout = cholesky_crout_pure(A)
L_scipy = scipy.linalg.cholesky(A_array, lower=True).tolist()

assert L_banachiewicz == L_crout == L_scipy
print "Looks good!"

Looks good!


Using the ``timeit`` module, let's write a small function that'll let us profile our various implementations:

In [3]:
import timeit

def profile(func, n=100000):
    def profiled_func(*args, **kwargs):
        total = 0.0
        worst = 0.0
        best = 999999.999 # a sufficiently large amount of time
        for _ in xrange(n):
            start_time = timeit.default_timer()
            func(*args, **kwargs)
            end_time = timeit.default_timer()
            duration = end_time - start_time
            
            if duration > worst:
                worst = duration
            if duration < best:
                best = duration
            
            total += duration
        avg = total / n
        
        print "%s:" % (func.__name__)
        print "  average execution time = %f" % avg
        print "  fastest execution time = %f" % best
        print "  slowest execution time = %f" % worst

    return profiled_func

Alternatively, we could make use of the ``%timeit`` line magic:
```
%timeit -r 10 cholesky_banachiewicz_pure(A)
```
For the sake of clarity, however, we'll be using our small ``profile`` function to benchmark our code for the rest of the tutorial.

In [4]:
profile(cholesky_banachiewicz_pure)(A)
profile(cholesky_crout_pure)(A)
profile(scipy.linalg.cholesky)(A_array, lower=True)

cholesky_banachiewicz_pure:
  average execution time = 0.000022
  fastest execution time = 0.000014
  slowest execution time = 0.005407
cholesky_crout_pure:
  average execution time = 0.000021
  fastest execution time = 0.000014
  slowest execution time = 0.003518
cholesky:
  average execution time = 0.000015
  fastest execution time = 0.000011
  slowest execution time = 0.000489


It looks like the SciPy implementation is a few microseconds faster than our pure Python implementation. As the matrices get larger, so does the difference in execution time between pure Python and SciPy. To illustrate, I've included some graphs of what the execution time looks like as the size of $A$ grows:

![SciPy vs. Pure Python](figures/scipy_vs_pure.png)

Let's see how these performance graphs change when we optimize our code using Numba.

In [5]:
from numba import jit
import numpy as np

@jit
def cholesky_banachiewicz_numba(A):
    n = len(A)
    L = np.zeros(A.shape)
    
    for i in xrange(n):
        for j in xrange(i + 1):
            temp = A[i,j]
            for k in xrange(j):
                temp -= L[i,k] * L[j,k]
                
            L[i,j] = temp**0.5 if i == j else temp / L[j,j]
            
    return L

@jit 
def cholesky_crout_numba(A):
    n = len(A)
    L = np.zeros(A.shape)
    
    for j in xrange(n):
        for i in xrange(j, n):
            temp = A[i,j]
            for k in xrange(j):
                temp -= L[i,k] * L[j,k]
            
            L[i,j] = temp**0.5 if i == j else temp / L[j,j]
    
    return L

In [6]:
profile(cholesky_banachiewicz_numba)(A_array)
profile(cholesky_crout_numba)(A_array)
profile(scipy.linalg.cholesky)(A_array, lower=True)

cholesky_banachiewicz_numba:
  average execution time = 0.000005
  fastest execution time = 0.000001
  slowest execution time = 0.403576
cholesky_crout_numba:
  average execution time = 0.000003
  fastest execution time = 0.000001
  slowest execution time = 0.184164
cholesky:
  average execution time = 0.000016
  fastest execution time = 0.000011
  slowest execution time = 0.002485


Now we're beating SciPy's implementation by a longshot. That's great! But wait, our code has changed. Why? Numba only accelerates code that uses scalars or (N-dimensional) arrays. You can't use built-in types like ``list`` or ``dict`` or your own custom classes, you can't allocate new arrays in accelerated code, and you can't even use recursion. This means that Numba is only useful in certain cases.

Let's see how the performance changes as we increase the size of the matrix:

![Numba vs. SciPy](figures/numba_vs_scipy.png)

For additional information, examples, and documentation, check out the [Numba](http://numba.pydata.org/) website. In general though, to Numba-fy your code, apply the ``@jit`` decorator, remove built-in types and custom classes, translate recursive functions to iterative ones, and don't alloacate new arrays if possible.


## The Cython Language
###### (Cython docs)
Cython is a programming language that makes writing C extensions for the Python language as easy as Python itself. It aims to become a superset of the Python language which gives it high-level, object-oriented, functional, and dynamic programming. Its main feature on top of these is support for optional static type declarations as part of the language. The source code gets translated into optimized C/C++ code and compiled as Python extension modules. This allows for both very fast program execution and tight integration with external C libraries, while keeping up the high programmer productivity for which the Python language is well known.

Using Cython in IPython notebooks is fairly straightforward. First, we load the Cython extension within our notebook:

In [7]:
%load_ext Cython

Now, we can use the cell magic ``%%cython`` to compile our original pure Python solution in the next cell:

In [8]:
%%cython
def cholesky_banachiewicz_cython_v1(A):
    n = len(A)
    L = [[0.0] * n for _ in xrange(n)]
    
    for i in xrange(n):
        for j in xrange(i + 1):
            temp = A[i][j] - sum(L[i][k] * L[j][k] for k in xrange(j))
            L[i][j] = temp**0.5 if i == j else temp / L[j][j]
            
    return L

def cholesky_crout_cython_v1(A):
    n = len(A)
    L = [[0.0] * n for _ in xrange(n)]
    
    for j in xrange(n):
        for i in xrange(j, n):
            temp = A[i][j] - sum(L[i][k] * L[j][k] for k in xrange(j))
            L[i][j] = temp**0.5 if i == j else temp / L[j][j]
    
    return L

In [9]:
profile(cholesky_banachiewicz_cython_v1)(A)
profile(cholesky_crout_cython_v1)(A)
profile(scipy.linalg.cholesky)(A_array, lower=True)

cholesky_banachiewicz_cython_v1:
  average execution time = 0.000012
  fastest execution time = 0.000009
  slowest execution time = 0.001289
cholesky_crout_cython_v1:
  average execution time = 0.000007
  fastest execution time = 0.000005
  slowest execution time = 0.000626
cholesky:
  average execution time = 0.000016
  fastest execution time = 0.000011
  slowest execution time = 0.001029


Notice how we only needed to use the ``%%cython`` cell magic to gain this speedup. Unlike with Numba, we didn't need to make any changes to our code to see improvements; however, the speedup we get from Cython isn't quite as good as the one we get from Numba. In fact, this approach yields almost no improvement for small matrices, and can actually worsen our performance as the dimensions of our input grows:

![Cython (v1) vs. Pure Python](figures/cython-v1_vs_pure.png)
![Cython (v1) vs. Numba](figures/cython-v1_vs_numba.png) 
![Cython (v1) vs. SciPy](figures/cython-v1_vs_scipy.png)

Not all hope is lost for Cython, though! We can do better using what are called *typed memoryviews* and learning a little bit more about the Cython language.

#### C vs. Python Functions/Variables in Cython
In Cython, we can declare both C variables/functions and Python variables/functions. To declare a C variable or function, we use the ``cdef`` keyword with type definitions. Python variables and functions can be declared just as they are in Python. If we wanted to declare integers ``i, j, k`` and floats ``f, g[42], *h`` as C variables, we would do the following
``` cython
cdef int i, j, k
cdef float f, g[42], *h
```
C functions written in Cython, like C variables, are declared with the ``cdef`` keyword. C functions in Cython can take in either Python objects or C values as arguments, and can return either Python objects or C values. The scope of C functions written is Cython is limited, however, to the module in which it was written: "Within a Cython module, Python functions and C functions can call each other freely, but only Python functions can be called from outside the module interpreted by Python code. So any functions that you want to "export" from your Cython module must be declared as Python functions using ``def``." 

To learn more about the differences between C functions and Python functions in Cython, check out [Cython Language Basics](http://cython.readthedocs.io/en/latest/src/userguide/language_basics.html).

#### Buffers and MemoryViews in Python
Before we continue, let's take a moment to consider how Python does operations on things like Strings, DataFrames, and Series. These objects (except for DataFrames and Series when inplace operations are applied) are immutable. To perform calculations and transformations on them require that we first make a copy of the object and then apply our operations. Whenever we index into a String, DataFrame, or Series by slicing, we're making a copy of the original object. This is why you'll notice that your program runs a lot slower when you, for example, define an empty DataFrame and iteratively insert all the rows one-by-one, than when you use a mutable class (like a dictionary) to iteratively build your object *and then* convert it to a DataFrame.

Python objects implemented in C can export a group of functions called the "buffer interface." These functions can be used by an object to expose its data in a raw byte-oriented format. Clients of the object can use the buffer interface to access the object data directly, without needing to copy it first. Since the release of Python 2.7, MemoryViews and Buffers provide an efficient way to deal with the general copying behavior when dealing with objects like Strings, DataFrames, and Series. A MemoryView is just like a buffer, except you can also write to it, not just read.

To learn more about MemoryViews in Python 2.7, check out the [documentation](https://docs.python.org/2/c-api/buffer.html).
  
#### Typed MemoryViews in Cython
From the Cython documentation page on [Typed MemoryViews](http://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html): "Typed MemoryViews allow efficient access to memory buffers, such as those underlying NumPy arrasys, without incurring any Python overhead." Here are some examples of using typed MemoryViews in Cython (taken from the documentation):
```cython
    # Create a complete view on a one-dimensional int buffer:
    cdef int[:] view1d = oneD_obj
    
    # A complete 3D view:
    cdef int[:, :, :] view3D = threeD_obj
    
```
Using Typed MemoryViews in our Cython code will provide the compiler more information about the desired behavior, enabling it to make further optimizations at compile-time.

Armed with this information, we can now create an improved version of our Cython implementation:

In [10]:
%%cython
import numpy as np
def cholesky_banachiewicz_cython_v2(long[:, :] A):
    cdef int i, j, k
    cdef int n = len(A)
    cdef double[:, :] L = np.zeros(shape=(n, n))
    
    for i in xrange(n):
        for j in xrange(i + 1):
            temp = A[i][j] - sum(L[i][k] * L[j][k] for k in xrange(j))
            L[i][j] = temp**0.5 if i == j else temp / L[j][j]
    
    return np.asarray(L)


def cholesky_crout_cython_v2(long[:, :] A):
    cdef int i, j, k
    cdef int n = len(A)
    cdef double[:, :] L = np.zeros(shape=(n, n))
    
    for j in xrange(n):
        for i in xrange(j, n):
            temp = A[i][j] - sum(L[i][k] * L[j][k] for k in xrange(j))
            L[i][j] = temp**0.5 if i == j else temp / L[j][j]
            
    return np.asarray(L)

In [11]:
profile(cholesky_banachiewicz_cython_v2)(A_array)
profile(cholesky_crout_cython_v2)(A_array)
profile(scipy.linalg.cholesky)(A_array, lower=True)

cholesky_banachiewicz_cython_v2:
  average execution time = 0.000013
  fastest execution time = 0.000009
  slowest execution time = 0.002615
cholesky_crout_cython_v2:
  average execution time = 0.000013
  fastest execution time = 0.000009
  slowest execution time = 0.001803
cholesky:
  average execution time = 0.000015
  fastest execution time = 0.000011
  slowest execution time = 0.001606


Now, the performance graphs look a little different:

![Cython (v2) vs. Pure Python](figures/cython-v2_vs_pure.png)
![Cython (v2) vs. Numba](figures/cython-v2_vs_numba.png) 
![Cython (v2) vs. SciPy](figures/cython-v2_vs_scipy.png)

We managed to dramatically improve our performance relative to our ``_cython_v1`` implementations; however, we didn't beat out Numba or SciPy. In any case, though, the difference between our two Cython implementations should provide sufficient evidence to convince you that simply using the ``%%cython`` magic isn't sufficient to make full use of Cython.

Using Cython in your terminal takes a little more work, but [this](http://cython.readthedocs.io/en/latest/src/quickstart/build.html) should show you how to get up and running.

## Conclusion
First and foremost, we should walk away from this being reassured in the efficiency of the existing SciPy implementations. Unless you're tackling something very specific, it's almost always a good idea to use the SciPy implementation if it's available to you. 

However, what do we do when we don't have SciPy available to us, or it's not exactly what we need? That's where Numba and Cython come in. Due to the limitations of Numba (e.g., no lists, dicts, recursion, custom classes, etc.), it's not always the appropriate solution. In the cases where we can't use Numba, we can use Cython, which allows us to gain some noticeable speedups in comparison to a pure Python implementation.

We've also seen that Cython's relative generalizability comes at a cost: it takes significantly more effort on behalf of the coder to reach Numba or SciPy-like levels of efficiency. At a certain point, the amount of effort spent optimizing a Cython function might be better spent writing an actual C implementation with considerations being made to memory layout and caching.